In [5]:
import os
import json
import cv2
import numpy as np
import matplotlib.pyplot as plt
from inference.infer import PoseDetector, ActionClassifier

In [23]:
with open('./data_folder/parameter/label_rule_value.json', 'rb') as r:
    s = json.load(r)

In [9]:
# list 타입인 file 변수 중 원하는 영상에 접근 하시면 됩니다. 
# list 슬라이싱 사용하시면 되여
# -------------------------------------------------------------------
os.chdir('teamproject_golf/')
files = os.listdir('data_folder/src/')
files
file = files[-3] # 이부분 변경하시면 되여 예) files[0] -> files[1]
src_dir = os.path.join('data_folder/src/', file)
src_dir
# 영상을 읽어 address, top 동작 등에 맞는 프레임을 찾아 
# 좌표들을 반환하는 코드입니다.

'data_folder/src/sm3.mp4'

In [10]:
# ----------------------------------------------------------
pose = PoseDetector()
ac = ActionClassifier()
with open('data_folder/parameter/params.json', 'r') as r:
    params = json.load(r)

In [ ]:
actions = params.keys()
for action in actions:
    video = cv2.VideoCapture(src_dir)
    while video.isOpened():
        read_ok, frame = video.read()
        if not read_ok:
            print('TRK Complete!')
            break
        cnt = video.get(cv2.CAP_PROP_POS_FRAMES)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, dsize=(0, 0), fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
        pose_obj = pose.trk_process(image=frame)
        pose.get_coordis_3d(pose_obj)
        landmarked_img = pose.drawing_with_pose(frame, pose_obj)
        ac.update_information(action=action,
                            parameter=params,
                            pose_coordis=pose.coordinates,
                            num_frame=cnt, image=landmarked_img)
for action in actions:
    video = cv2.VideoCapture(src_dir)
    while video.isOpened():
        read_ok, frame = video.read()
        if not read_ok:
            print('TRK Complete!')
            break
        cnt = video.get(cv2.CAP_PROP_POS_FRAMES)
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, dsize=(0, 0), fx=1.5, fy=1.5, interpolation=cv2.INTER_LINEAR)
        pose_obj = pose.trk_process(image=frame)
        pose.get_coordis_3d(pose_obj)
        landmarked_img = pose.drawing_with_pose(frame, pose_obj)
        ac.update_information(action=action,
                            parameter=params,
                            pose_coordis=pose.coordinates,
                            num_frame=cnt, image=landmarked_img)
action

In [ ]:
#### 데이터 설명 
 - 위 셀이 실행 되면 ac.esti_inform 속성으로 데이터가 저장됩니다.
 - ac.esti_inform의 데이터 구성은 아래와 같습니다.
 - {동작 구분 : {닮음 정도, 좌표, 이미지}}
ac.esti_inform
plt.figure(figsize=(15,15))
plt.imshow(ac.esti_inform['top']['image'])
# address 동작의 좌표들에 접근하기 위해서는 아래 코드와 같이 접근합니다.
# ac.esti_inform['address']['coordinate'] -> dictionary 타입
# ------------------------------------------------------------------
ac.esti_inform['backswing']['coordinate']
# 위 데이터는 각 포인트별로 엉덩이 가운데를 기준으로 얼마나 떨어져 있는지를 x, y, z
# 좌표값으로 줍니다
# 예시) ac.esti_inform['address']['coordinate']['nose']  
#              └> 어드레스 동작중 nose의 x, y, z 값(np.ndarray 형태)  
# ---------------------------------------------------------------------------------
ac.esti_inform['backswing']['coordinate']['left_wrist']
ac.esti_inform['address']['coordinate']
ac.esti_inform['backswing']['coordinate']
ac.esti_inform['top']['coordinate']
ac.esti_inform['impact']['coordinate']
ac.esti_inform['follow']['coordinate']

In [11]:
# 두 점으로 기울어진 정도를 구하는 함수
def tan_slope(a:np.ndarray,b:np.ndarray) -> float:
    '''두 Points 가 주어졌을 때 두 point간 vector를 구하고 vector와 x축간 기울어진 정도를 구하는 함수
    **args**
    - a : 첫 번째 point의 vector(np.ndarray) 
    - b : 두 번째 point의 vector(np.ndarray) 
    **return**
    - vector와 x축간 기울어진 정도(float, 0~180)'''
    
    x = b[0] - a[0]
    y = abs(b[1] - a[1])
    slope = abs(180/np.pi * np.arctan(y/x))
    return round(slope,2)

# 세점이 주어졌을 때 2번째 점에서의 각도를 구하는 함수
def angle(a:np.ndarray,b:np.ndarray,c:np.ndarray) -> float:
    '''세 Points 가 주어졌을 때 두번째 point에서 형성되는 각도를 구하는 함수
        **args**
        - a : 첫 번째 point의 vector(np.ndarray) 
        - b : 두 번째 point의 vector(np.ndarray)
        - c : 세 번째 point의 vector(np.ndarray)
        **return**
        - 두번째 point에서 형성되는 각도(float)'''
    v1 = c-b
    v2 = a-b
    v1 = v1[:2]
    v2 = v2[:2]
    angle_cos = (v1@ v2)/(np.linalg.norm(v1,2)*np.linalg.norm(v2,2))
    angle = 180/np.pi*np.arccos(angle_cos)
    return round(angle,2)

# 2점이 주어졌을 때 가상의 라인을 긋고 그에 대한 각도를 구하는 함수
def angle_2(a:np.ndarray,b:np.ndarray) -> float:
    '''두 Points 가 주어졌을 때 y축방향의 가상의 line을 긋고 가상의 line에서 가상의 point를 정하여 두번째 point에서 형성되는 각도를 구하는 함수
       가상의 point(new point)는 첫번째 점의 x좌표값과 두번째 점의 y좌표값으로 구한다
       **args**
        - a : 첫 번째 point의 vector(np.ndarray) 
        - b : 두 번째 point의 vector(np.ndarray)
        **return**
        - 두번째 point에서 형성되는 각도(float)'''
    new_point = [b[0],a[1],0]
    v1 = a-b
    v2 = new_point-b
    v1 = v1[:2]
    v2 = v2[:2]
    angle_cos = (v1@ v2)/(np.linalg.norm(v1,2)*np.linalg.norm(v2,2))
    angle = 180/np.pi*np.arccos(angle_cos)
    return round(angle,2)

def ratio(a:np.ndarray, b:np.ndarray, c:np.ndarray, d:np.ndarray, dim=3) -> float:
    '''4개 vectors가 주어졌을 때 첫번째 두번째 vectors pair와 세번째 네번째 vector pair간 비율을 구하는 함수
    **args**
    - (a,b) : 첫번째 vector 연산을 위한 2개 vectors(np.ndarray) 
    - (c,d) : 두번째 vector 연산을 위한 2개 vectors(np.ndarray)
    - dim : 계산하고자 하는 차원수
    **return**
    - 첫번째 vector 대비 두번째 vector 비율(float)'''
    
    r1_vector = c[:dim] - d[:dim]
    r1_norm = np.linalg.norm(r1_vector,2)
    r2_vector = a[:dim] - b[:dim]
    r2_norm = np.linalg.norm(r2_vector,2)
    return round(abs(r2_norm/r1_norm*100),2)

def ratio_interpolation(a:np.ndarray,b:np.ndarray,cal_point:np.ndarray):
    '''3개 vectors가 주어졌을 때 첫번째 두번째 vectors pair와 cal_point와 첫번째 vector간 비율을 구하는 함수
        **args**
        - (a,b) : vector 연산을 위한 2개 vectors(np.ndarray) 
        - cal_point : interpolation point
        - dim : 계산하고자 하는 차원수
        **return**
        - 첫번째 vector 대비 두번째 vector 비율(float)
    '''
    r1_vector = a[0] - b[0]
    r2_vector = a[0] - cal_point[0]
    print(r1_vector)
    print(r2_vector)
    print(cal_point)
    return round(abs(r2_vector/r1_vector*100),2)

def center_calc(start:np.ndarray, end:np.ndarray):
    res = start + ((end-start)*0.5)
    return res

In [13]:
os.listdir('./data_folder/labels/jsons/')

['address.json', 'backswing.json', 'follow.json', 'impact.json', 'top.json']

In [18]:
inform = {}
for action in os.listdir('./data_folder/labels/jsons/'):
    ac = action[:-5]
    with open(os.path.join('./data_folder/labels/jsons/', action), 'rb') as r:
        inform[ac] = json.load(r)

In [ ]:
## 1. address
### 어깨선각도
# 오른쪽어깨(12)~왼쪽어깨(11)간 각도
tan_slope(ac.esti_inform['address']['coordinate']['left_shoulder'],ac.esti_inform['address']['coordinate']['right_shoulder'])
### 양손위치
# 왼쪽골반(23)~오른쪽골반(24) : 왼쪽손목(15), 오른쪽손목(16)
res = center_calc(ac.esti_inform['address']['coordinate']['left_wrist'],ac.esti_inform['address']['coordinate']['right_wrist'])
ratio_interpolation(ac.esti_inform['address']['coordinate']['left_ankle'],ac.esti_inform['address']['coordinate']['right_ankle'],res)
### 스탠스
# 왼발(29), 오른발(30) : 오른쪽어깨(12)~왼쪽어깨(11)
ratio(ac.esti_inform['address']['coordinate']['left_shoulder'],ac.esti_inform['address']['coordinate']['right_shoulder'],ac.esti_inform['address']['coordinate']['left_heel'],ac.esti_inform['address']['coordinate']['right_heel'])
### 상체기울임정도
# 하체비율(30~24) : 상체비율(24~12)
ratio(ac.esti_inform['address']['coordinate']['right_heel'],ac.esti_inform['address']['coordinate']['right_hip'],ac.esti_inform['address']['coordinate']['right_hip'],ac.esti_inform['address']['coordinate']['right_shoulder'])

In [ ]:
## 2.백스윙
### 왼쪽 어깨 회전
# 왼발(29), 오른발(30) : 왼쪽어깨(11) 
ratio_interpolation(ac.esti_inform['backswing']['coordinate']['right_ankle'],ac.esti_inform['backswing']['coordinate']['left_ankle'],ac.esti_inform['top']['coordinate']['left_shoulder'])
### 왼팔 펴짐 각도
# 왼쪽 어깨(11) ~ 왼쪽 팔꿈치(13) ~ 왼쪽 팔꿈치(15) 각도 
angle(ac.esti_inform['backswing']['coordinate']['left_wrist'],ac.esti_inform['backswing']['coordinate']['left_elbow'],ac.esti_inform['backswing']['coordinate']['left_shoulder'])
tan_slope(ac.esti_inform['backswing']['coordinate']['left_elbow'],ac.esti_inform['backswing']['coordinate']['left_shoulder'])
tan_slope(ac.esti_inform['backswing']['coordinate']['left_wrist'],ac.esti_inform['backswing']['coordinate']['left_elbow'])

In [ ]:
## 3. Top
### 하체 고정
# 오른쪽 골반(24) ~ 오른쪽 무릎(26) ~ 왼쪽 무릎(25) 각도
angle(ac.esti_inform['top']['coordinate']['right_knee'],ac.esti_inform['top']['coordinate']['right_hip'],ac.esti_inform['top']['coordinate']['left_knee'])
angle(ac.esti_inform['top']['coordinate']['right_hip'],ac.esti_inform['top']['coordinate']['right_knee'],ac.esti_inform['top']['coordinate']['left_knee'])
tan_slope(ac.esti_inform['top']['coordinate']['right_knee'],ac.esti_inform['top']['coordinate']['right_hip'])
tan_slope(ac.esti_inform['top']['coordinate']['right_knee'],ac.esti_inform['top']['coordinate']['left_knee'])
### 오른쪽 다리펴짐 각도
# 오른발(28)~오른쪽 무릎(26) ~ 오른쪽 골반(24) 각도
angle(ac.esti_inform['top']['coordinate']['right_hip'],ac.esti_inform['top']['coordinate']['right_knee'],ac.esti_inform['top']['coordinate']['right_ankle'])
### 스웨이 체크
res = center_calc(ac.esti_inform['top']['coordinate']['left_shoulder'],ac.esti_inform['top']['coordinate']['right_shoulder'])
ratio_interpolation(ac.esti_inform['top']['coordinate']['right_ankle'],ac.esti_inform['top']['coordinate']['left_ankle'],res)
ac.esti_inform['top']['coordinate']['right_ankle']
ac.esti_inform['top']['coordinate']['left_ankle']
ac.esti_inform['top']['coordinate']['left_shoulder']
ac.esti_inform['top']['coordinate']['right_shoulder']
0.32496628-(-0.2461912)
0.07465049-(-0.2461912)
(0.32084169)/0.57115748

In [ ]:
## 4. Impact
### 어깨선 각도(그냥 구해봄)
# 오른쪽어깨(12)~왼쪽어깨(11)간 각도
tan_slope(ac.esti_inform['impact']['coordinate']['left_shoulder'],ac.esti_inform['impact']['coordinate']['right_shoulder'])
### 행잉백
# 왼쪽어깨(11)~(왼쪽발목(x), 왼쪽어깨(y)점(*파생))~왼쪽발목(27)
angle_2(ac.esti_inform['impact']['coordinate']['left_shoulder'],ac.esti_inform['impact']['coordinate']['left_ankle'])
### 오른쪽 K라인 각도
# 오른쪽어깨(12)~오른쪽골반(24)~오른쪽발목(28)
angle(ac.esti_inform['impact']['coordinate']['right_shoulder'],ac.esti_inform['impact']['coordinate']['right_hip'],ac.esti_inform['impact']['coordinate']['right_ankle'])
### 왼팔 펴짐 각도
# 왼쪽 어깨(11) ~ 왼쪽 팔꿈치(13) ~ 왼쪽 팔꿈치(15) 각도 
angle(ac.esti_inform['impact']['coordinate']['left_wrist'],ac.esti_inform['impact']['coordinate']['left_elbow'],ac.esti_inform['impact']['coordinate']['left_shoulder'])
### 오른팔 펴짐 각도
# 오른쪽 어깨(12) ~ 오른쪽 팔꿈치(14) ~ 오른쪽 팔꿈치(16) 각도 
angle(ac.esti_inform['impact']['coordinate']['right_wrist'],ac.esti_inform['impact']['coordinate']['right_elbow'],ac.esti_inform['impact']['coordinate']['right_shoulder'])

In [ ]:
## 5.팔로우스루
### 왼쪽 라인
# 왼쪽발목(27)~(왼쪽발목(x), 왼쪽골반(y),*파생)~왼쪽골반(23)
angle_2(ac.esti_inform['follow']['coordinate']['left_hip'],ac.esti_inform['follow']['coordinate']['left_ankle'])
### 치킨윙
# 왼쪽 어깨(11) ~ 왼쪽 팔꿈치(13) ~ 왼쪽 팔꿈치(15) 각도 
angle(ac.esti_inform['follow']['coordinate']['left_wrist'],ac.esti_inform['follow']['coordinate']['left_elbow'],ac.esti_inform['follow']['coordinate']['left_shoulder'])
### 체중 전진 이동
# 왼발(29), 오른발(30) : 원점(골반중심)
ratio_interpolation(a=ac.esti_inform['follow']['coordinate']['right_ankle'],
                    b=ac.esti_inform['follow']['coordinate']['left_ankle'],
                    cal_point=np.array([0,0,0]))
### 오른쪽 다리 펴짐 각도
# 오른발(28)~오른쪽 무릎(26) ~ 오른쪽 골반(24) 각도
angle(ac.esti_inform['follow']['coordinate']['right_hip'],ac.esti_inform['follow']['coordinate']['right_knee'],ac.esti_inform['follow']['coordinate']['right_ankle'])